# Descarga de datos del fichero de Indicadores de Seguimiento publicado por Ministerio de Sanidad

## Objetivo

Vamos a hacer una lectura de los datos publicados en el informe https://t.co/KOFUAhUynL?amp=1 publicado por Sanidad el 30 Noviembre:


Manuel H. Arias  
[@walyt](https://twitter.com/walyt)  

[#escovid19data](https://github.com/montera34/escovid19data)



## Código

Como siempre importamos las librerías con las que vamos a trabajar:

In [73]:
import os.path as pth
import datetime as dt
import time
from glob import glob
import re
import pandas as pd
import numpy as np

import requests
from shutil import copyfile

import matplotlib.pyplot as plt
from matplotlib.ticker import FuncFormatter
from matplotlib import cm
import matplotlib.dates as mdates
import matplotlib.ticker as ticker
from matplotlib.dates import (YEARLY, MONTHLY, DateFormatter, WeekdayLocator, MonthLocator,DayLocator,
                              rrulewrapper, RRuleLocator, drange)
import seaborn as sns
import matplotlib.colors as colors

from pdfminer.pdfinterp import PDFResourceManager, PDFPageInterpreter
from pdfminer.pdfpage import PDFPage
from pdfminer.converter import TextConverter
from pdfminer.layout import LAParams
from io import StringIO


import numpy as np
from datetime import datetime,timedelta
import seaborn as sns
%matplotlib inline

Definimos variables que nos ayuden en la gestión de los nombres de los ficheros, nombres de columnas e indices. 

In [74]:
datadir='datos_sanidad_seguimiento/'
URL_reg='https://www.mscbs.gob.es/profesionales/saludPublica/ccayes/alertasActual/nCov/documentos/informe_covid_es_publico_{}.pdf'

In [75]:
# cada una de las zonas de las que nos da datos, se ha generado con un cut&paste 
# desde el pdf con tratamiento posterior
zonas=['Coruña','Alacant/Alicante','Albacete','Almería','Araba/Álava','Asturias',
            'Ávila','Badajoz','Barcelona','Bizkaia','Burgos','Cáceres','Cádiz',
            'Cantabria','Castelló/Castellón','Ceuta','CiudadReal','Córdoba','Cuenca',
            'ElHierro','Formentera','Fuerteventura','Gipuzkoa','Girona','Gomera',
            'Granada','GranCanaria','Guadalajara','Huelva','Huesca','Ibiza','Jaén',
            'Lanzarote','LaPalma','LaRioja','León','Lleida','Lugo','Madrid','Málaga',
            'Mallorca','Melilla','Menorca','Murcia','Navarra','Ourense','Palencia',
            'Pontevedra','Salamanca','Segovia','Sevilla','Soria','Tarragona','Tenerife',
            'Teruel','Toledo','València/Valencia','Valladolid','Zamora','Zaragoza']

In [76]:
#nombre cada columna, de forma simplificadad
columnas=['IA14','IA7','IA14_65','IA7_65','positividad','trazabilidad','ocupacion_agudos','ocupacion_UCI']

In [77]:
#generamos un pandas vacío con esas columnas e indice tal que las zonas
indicadores=pd.DataFrame(index=zonas,columns=columnas)

In [78]:
indicadores.head(5)

,IA14,IA7,IA14_65,IA7_65,positividad,trazabilidad,ocupacion_agudos,ocupacion_UCI
Coruña,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Alacant/Alicante,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Albacete,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Almería,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Araba/Álava,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### Función para descargar un fichero pdf

Definimos a continuación una función para descargar el fichero pdf de web

In [79]:
def descarga(url,guardarlo_como):
    '''
    Devuelve True si éxito descargando el pdf del link url, grabándolo como guardarlo_como
    Parametros:
        url: url del fichero a descargar
        guardarlo_como : nombre del fichero a grabar, sin extensión.
    '''
    print('Descargando:', url)
    nombre_a_guardar=guardarlo_como+'.pdf'
    headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_10_1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/39.0.2171.95 Safari/537.36'}
    with requests.Session() as s:
        r = s.get(url, headers=headers)
    if r.status_code == requests.codes.ok:
        with open(nombre_a_guardar, 'wb') as fp:
                fp.write(r.content)
                print('Guardándolo como:', nombre_a_guardar)
                return True
    else:
        print ('Error con el ',nombre_a_guardar)
        return False

 Vamos a llamar a los ficheros tal que su día de publicación en formato `%Y-%m-%d`, "2020-12-05" por ejemplo...

In [80]:
un_dia=timedelta(days=1)
hoy=datetime.today()
#descargamos el del 30-11
fecha_str='2020-11-30'

In [81]:
descarga(URL_reg.format(fecha_str),datadir+fecha_str)

Descargando: https://www.mscbs.gob.es/profesionales/saludPublica/ccayes/alertasActual/nCov/documentos/informe_covid_es_publico_2020-11-30.pdf
Guardándolo como: datos_sanidad_seguimiento/2020-11-30.pdf


True

### Extraemos el texto de la pagina concreta del pdf

In [82]:
# Extract PDF text using PDFMiner. Adapted from
# http://stackoverflow.com/questions/5725278/python-help-using-pdfminer-as-a-library
# codigo copiado del script de @alfonsotwr
# desde 
def pdf_to_text(pdfname, pagenum=None):
    '''
    Devuelve el texto de la página pagenum extraído del pdf de path=pdfname
    Parametros:
        pdfname:pdf del fichero pdf
        pagenum : página del fichero
    '''
    # PDFMiner boilerplate
    rsrcmgr = PDFResourceManager()
    sio = StringIO()
    laparams = LAParams()
    device = None
    try:
        device = TextConverter(rsrcmgr, sio, laparams=laparams)
        interpreter = PDFPageInterpreter(rsrcmgr, device)

        # Extract text
        with open(pdfname, 'rb') as fp:
            for i, page in enumerate(PDFPage.get_pages(fp)):
                if pagenum is None or pagenum == i:
                    interpreter.process_page(page)

        # Get text from StringIO
        text = sio.getvalue()
    finally:
        # Cleanup
        sio.close()
        if device is not None:
            device.close()

    return text

Tras estudiar el formato de `text` hemos generado unas RegEx para leer las series de números: basicamente se compone de una cabezera (IA x 100.000....) seguida de una lista de 30 números ó `NA`..

In [83]:
lista_expresion=['(IA x \n100.000 \n\(14 días\))(( \n\d*[,.]?\d+| \nNA){30})',
                 '(IA x \n100.000 \n\(7 días\))(( \n\d*[,.]?\d+| \nNA){30})',
                 '(IA >65 \naños x \n100.000 \n\(14 días\))(( \n\d*[,.]?\d+| \nNA){30})',
                 '(IA >65 \naños x \n100.000 \n\(7 días\))(( \n\d*[,.]?\d+| \nNA){30})',
                 '(positividad \nen PDIA\*\*)(( \n\d*[,.]?\d+| \nNA){30})',
                 '(casos \ncon \ntrazabilidad)(( \n\d*[,.]?\d+| \nNA){30})',
                 '(ocupación \ncamas de \nagudos por \nCOVID)(( \n\d*[,.]?\d+| \nNA){30})',
                 '(ocupación \ncamas de \nUCI por \nCOVID)(( \n\d*[,.]?\d+| \nNA){30})']

Empezamos con la página 3..

In [84]:
nombre_pdf=datadir+fecha_str+'.pdf'

#sacamos la tabla de la página 3, aprox la mitad de las zonas
text = pdf_to_text(nombre_pdf, pagenum=4)  #con que pagina queremos trabajar?
#iteramos por cadad una de las expresiones de lista_expresion   
for i,expresion in enumerate(lista_expresion):
    match=re.search(re.compile(r'{}'.format(expresion)),text)
    datos=list(match.group(2).replace('\n','').split(' '))[1:]#eliminamos el primer elemento
    indicadores.loc[zonas[0]:zonas[len(datos)-1],columnas[i]]=datos
    print (columnas[i],'-->',datos)
                         

IA14 --> ['211.0', '250.1', '275.7', '328.5', '317.6', '391.4', '154.1', '205.5', '234.1', '344.6', '698.1', '262.6', '377.5', '326.8', '236.2', '280.7', '273.5', '262.7', '477.8', 'NA', '57.8', '39.4', '518.8', '300.9', '60.5', '406.4', '36.1', '197.5', '362.4', '368.8']
IA7 --> ['92.8', '108.2', '117.0', '139.2', '142.7', '164.2', '55.8', '90.6', '101.7', '141.7', '242.6', '117.7', '157.7', '124.1', '99.5', '88.5', '122.4', '111.8', '192.5', 'NA', '8.3', '22.2', '210.3', '141.4', '51.2', '133.5', '17.9', '88.1', '145.8', '120.7']
IA14_65 --> ['188.8', '226.2', '293.2', '252.2', '222.0', '496.9', '115.7', '167.0', '241.0', '311.7', '732.1', '371.1', '357.5', '340.5', '162.4', '0.0', '226.3', '258.9', '452.5', 'NA', '155.1', '49.7', '478.7', '326.9', '79.0', '382.7', '31.5', '228.5', '258.8', '268.0']
IA7_65 --> ['93.7', '99.4', '125.3', '113.0', '112.4', '203.9', '44.3', '81.6', '111.0', '120.8', '290.3', '188.3', '161.7', '128.2', '70.2', '0.0', '106.2', '132.4', '192.6', 'NA', '0.0'

..y vamos con el resto de datos que están en la siguiente página. Esta página tiene las listas consecutivas, divididas solo por un \n. Vale pues con una sola expresión, si embargo hemos de iterar sobre las diferentes coincidencias:

In [85]:
expresion='(\n)?((\n\d*[,.]?\d+ |\nNA ){30})'
fn=datadir+fecha_str+'.pdf'
fn1 = fn.replace('.pdf', '.txt')
page1 = pdf_to_text(fn, pagenum=5)  #con que pagina queremos trabajar?
    #with open(fn1, 'w', encoding='utf-8') as fp:
with open(fn1, 'w') as fp:    
    fp.write(page1)
with open(fn1) as fp:
    text = fp.read()
    #lista=text.partition(texto1)[2].partiion(texto1)[2].partition(texto1)[2].replace('\n','').split(' ')

for i,match in enumerate(re.finditer(re.compile(r'{}'.format(expresion)),text)):
    datos=list(match.group(2).replace('\n','').split(' '))[:-1]
    indicadores.loc[zonas[len(datos)]:,columnas[i]]=datos
    print (datos)

['149.0', '412.7', '36.0', '25.9', '389.8', '363.7', '365.8', '228.2', '227.6', '208.0', '230.6', '404.7', '89.9', '252.4', '264.1', '117.3', '536.1', '290.8', '253.8', '175.0', '249.2', '393.7', '189.4', '132.5', '396.6', '311.7', '307.6', '448.3', '472.4', '311.3']
['61.3', '150.6', '19.2', '7.0', '175.5', '136.7', '175.2', '84.0', '108.1', '88.2', '122.3', '156.1', '56.7', '97.0', '115.7', '41.9', '209.3', '120.6', '84.5', '94.0', '80.5', '132.0', '77.9', '66.1', '223.7', '141.9', '129.5', '139.2', '168.7', '143.7']
['104.6', '377.1', '15.5', '33.9', '357.5', '341.6', '512.4', '278.4', '175.3', '214.8', '191.3', '354.5', '67.0', '194.1', '186.9', '133.1', '808.3', '313.6', '247.4', '188.1', '231.5', '479.3', '223.6', '111.6', '250.1', '367.7', '240.6', '451.9', '423.4', '271.0']
['65.4', '160.1', '15.5', '16.9', '175.0', '140.5', '262.6', '73.7', '82.3', '90.6', '96.0', '187.7', '46.9', '66.1', '68.9', '47.5', '333.9', '126.4', '82.9', '127.3', '71.6', '177.5', '108.9', '62.3', '148

Ya disponemos del pandas `indicadores` con todos los valores. Previo guardarlo en u fichero `csv` para compartir nos aseguraremos de que todos los valores tienen tipo numérico:

In [86]:
indicadores.applymap(lambda x: np.NaN if x=='NA' else eval(x))

,IA14,IA7,IA14_65,IA7_65,positividad,trazabilidad,ocupacion_agudos,ocupacion_UCI
Coruña,211.0,92.8,188.8,93.7,7.2,69.0,5.1,8.8
Alacant/Alicante,250.1,108.2,226.2,99.4,13.7,64.6,8.2,26.1
Albacete,275.7,117.0,293.2,125.3,12.1,70.5,8.0,33.3
Almería,328.5,139.2,252.2,113.0,9.8,58.8,7.3,23.4
Araba/Álava,317.6,142.7,222.0,112.4,7.0,6.3,13.0,26.0
Asturias,391.4,164.2,496.9,203.9,5.7,32.8,23.2,38.1
Ávila,154.1,55.8,115.7,44.3,5.3,69.3,12.9,47.4
Badajoz,205.5,90.6,167.0,81.6,9.6,78.0,8.0,6.2
Barcelona,234.1,101.7,241.0,111.0,5.6,42.4,10.6,31.4
Bizkaia,344.6,141.7,311.7,120.8,7.6,4.8,14.8,27.9


Guardamos el pandas como fichero `csv`:

In [87]:
indicadores.to_csv('indicadores_30112020.csv')